In [1]:
import pandas
import numpy
import datetime
import feather
# import the lshell fit calc module
import os
import sys
module_path = os.path.abspath(os.path.join('/home/bharat/Documents/code/indsvels/py/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import saps_lshell_vel_map

In [2]:
# Some constants
eventDate = datetime.datetime(2015,4,9)
timeStart = datetime.datetime(2015,4,9,4,0)
timeEnd = datetime.datetime(2015,4,9,8,0)
sapsTime = datetime.datetime(2015,4,9,6,30)
sapsRadList = [ "cvw", "cve","fhw",\
               "fhe", "bks", "wal" ]
selBeam = 7
sapsMlatRange = [ 55., 66. ]#[ 56., 60. ]
sapsVelCutoff = 50.
sapsNpointsCutoff = 0 # per radar per beam
velScale = [ -500, 500 ]
velMagnScale = [ 0, 2000 ]

In [3]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20150409,600,0,12,-49.7408,1.3438,41.5507,-151.5183,15.0557,46.5437,136.5005,41,hkw,2015-04-09 06:00:00,-8.9443
1,20150409,600,0,13,-50.5087,4.3482,41.6742,-151.8589,15.0330,46.6557,136.1289,41,hkw,2015-04-09 06:00:00,-8.9670
2,20150409,600,0,14,-51.2460,4.8619,41.7876,-152.1900,15.0109,46.7577,135.7681,41,hkw,2015-04-09 06:00:00,-8.9891
3,20150409,600,0,15,-51.8414,2.3704,42.0157,-152.6595,14.9796,46.9703,135.2588,41,hkw,2015-04-09 06:00:00,-9.0204
4,20150409,600,0,16,-52.4091,1.6610,42.2541,-153.1491,14.9470,47.1921,134.7289,41,hkw,2015-04-09 06:00:00,-9.0530


In [4]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF["velMagn"] = numpy.abs(velsDF["vLos"])
# setup some fake vals
velsDF["spwdth"] = 50.
velsDF["pwr"] = 10.
# we dont need fits for all the times
velsDF = velsDF[ (velsDF["date"] >= timeStart) &\
               (velsDF["date"] <= timeEnd) ]
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,velMagn,spwdth,pwr
0,20150409,600,0,12,-49.7408,1.3438,41.5507,-151.5183,15.0557,46.5437,136.5005,41,hkw,2015-04-09 06:00:00,-8.9443,1.3438,50.0,10.0
1,20150409,600,0,13,-50.5087,4.3482,41.6742,-151.8589,15.0330,46.6557,136.1289,41,hkw,2015-04-09 06:00:00,-8.9670,4.3482,50.0,10.0
2,20150409,600,0,14,-51.2460,4.8619,41.7876,-152.1900,15.0109,46.7577,135.7681,41,hkw,2015-04-09 06:00:00,-8.9891,4.8619,50.0,10.0
3,20150409,600,0,15,-51.8414,2.3704,42.0157,-152.6595,14.9796,46.9703,135.2588,41,hkw,2015-04-09 06:00:00,-9.0204,2.3704,50.0,10.0
4,20150409,600,0,16,-52.4091,1.6610,42.2541,-153.1491,14.9470,47.1921,134.7289,41,hkw,2015-04-09 06:00:00,-9.0530,1.6610,50.0,10.0


In [5]:
allTimeList = velsDF["date"].unique()
fitResFrames = []
failedDTObjs = []
for timeSel in allTimeList:
    print timeSel
    inpVelDF = velsDF[ (velsDF["date"] == timeSel)\
                     ].reset_index(drop=True)
    lmObj = saps_lshell_vel_map.LshellMap( inpVelDF, timeSel )
    # get locations info for getting good fits
    azimCharDF = lmObj.azim_chars()
    # If no good fit is found discard and store the 
    # datetimes in a list for reference!
    if azimCharDF.shape[0] == 0:
        failedDTObjs.append(timeSel)
        print "no good azims found--->", timeSel
        continue
    # get actual good fits
    goodFitDF = lmObj.get_good_fits(azimCharDF)
    print "good fits", goodFitDF.shape
    # If no good fits are found! exit
    if goodFitDF.shape[0] == 0:
        print "no good fits found--->", timeSel
        continue
    # expand the fitting to cells with no fits
    fitResDF = lmObj.expand_fit_results(goodFitDF)
    print "expanding good fits", fitResDF.shape
    # Set up a few conditions to filter out some data
    # 1) If the number of fits is less than 5 skip
    if fitResDF.shape[0] < 5:
        continue
    # 2) number of good fits should be greater than bad one's
    if (fitResDF[ fitResDF["goodFit"] \
            ].shape[0]*1./fitResDF.shape[0]) <= 0.5:
        print "too many bad fits"
        continue
    # 3) Number of unique MLTs in the fits should be greater
    # than or equal to  (for example, 0, 1)
#     if len( fitResDF["normMlt"].unique().tolist() ) < 2.:
#         print "no large scale observations...."
#         continue
    # Need the datetime col as well
    fitResDF["date"] = timeSel
    fitResFrames.append( fitResDF )
# Merge all the good fit DFs
finFitDF = pandas.concat( fitResFrames )

2015-04-09T06:00:00.000000000


/home/bharat/Documents/code/indsvels/py/saps_lshell_vel_map.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfSel["rndAzim"] = dfSel[self.fitAzmType].round()
/home/bharat/Documents/code/indsvels/py/saps_lshell_vel_map.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x in dfSel["normMLT"] ]


good fits (24, 8)
expanding good fits (45, 10)
2015-04-09T06:02:00.000000000
good fits (23, 8)
expanding good fits (40, 10)
2015-04-09T06:04:00.000000000
good fits (26, 8)
expanding good fits (48, 10)
2015-04-09T06:06:00.000000000
good fits (36, 8)
expanding good fits (58, 10)
2015-04-09T06:08:00.000000000
good fits (45, 8)
expanding good fits (62, 10)
2015-04-09T06:10:00.000000000
good fits (49, 8)
expanding good fits (68, 10)
2015-04-09T06:12:00.000000000
good fits (56, 8)
expanding good fits (73, 10)
2015-04-09T06:14:00.000000000
good fits (67, 8)
expanding good fits (80, 10)
2015-04-09T06:16:00.000000000
good fits (71, 8)
expanding good fits (90, 10)
2015-04-09T06:18:00.000000000
good fits (75, 8)
expanding good fits (95, 10)
2015-04-09T06:20:00.000000000
good fits (74, 8)
expanding good fits (90, 10)
2015-04-09T06:22:00.000000000
good fits (75, 8)
expanding good fits (95, 10)
2015-04-09T06:24:00.000000000
good fits (71, 8)
expanding good fits (95, 10)
2015-04-09T06:26:00.000000000

In [6]:
finFitDF['goodFit'] = [ 1 if x else 0 for x in finFitDF['goodFit'] ]
feather.write_dataframe( finFitDF, \
            "../data/lshellFits-" +\
            eventDate.strftime("%Y%m%d") + ".feather" )
print "File Created!"

File Created!
